In [1]:
%cd ..

/home/zaccharie/workspace/understanding-unets


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
import glob
import math
import os.path as op

from keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm_notebook

Using TensorFlow backend.


# Model loading

In [3]:
run_id = 'big_no_nlin'
load_path = op.join('saved_models', f'{run_id}.h5')
model = load_model(load_path)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [4]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 2)  20          input_1[0][0]                    
__________________________________________________________________________________________________
average_pooling2d_1 (AveragePoo (None, 128, 128, 2)  0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 128, 128, 4)  76          average_pooling2d_1[0][0]        
__________________________________________________________________________________________________
average_po

# Data iterates

In [5]:
bsd_dir_test = 'BSDS300/images/test'
bsd_dir_train = 'BSDS300/images/train'

In [6]:
filelist = glob.glob(bsd_dir_test + '/*') + glob.glob(bsd_dir_train + '/*')

In [14]:
im_list = [plt.imread(fname).mean(axis=-1) for fname in filelist]
noisy_im_list = [im + np.random.normal(scale=30, size=im.shape) for im in im_list]

noise_mses = [np.mean((noisy_im / 255 - im / 255)**2) for im, noisy_im in zip(im_list, noisy_im_list)]
print(np.mean(noise_mses))
print(np.mean(-10*np.log10(noise_mses)))

0.013837017787365638
18.589603273819655


In [8]:
def evaluate(im_noisy, im_gt):
    x_recon = denoise(im_noisy)
    return np.mean((x_recon - im_gt)**2)

def denoise_image(im_noisy):
    sub_patches = {}
    recon_matrix = np.zeros_like(im_noisy)
    for i in range(math.ceil(im_noisy.shape[0] / 256)):
        for j in range(math.ceil(im_noisy.shape[1] / 256)):
            i_max = min((i+1) * 256, im_noisy.shape[0]+1)
            i_min = max(0, i_max - 257)
            j_max = min((j+1) * 256, im_noisy.shape[1]+1)
            j_min = max(0, j_max - 257)
            slice_i = slice(i_min, i_max)
            slice_j = slice(j_min, j_max)
            recon_matrix[slice_i, slice_j] += 1
            sub_patches[(i, j)] = im_noisy[slice_i, slice_j]
    sub_patches_done = {k: model.predict(sub_patch[None, ..., None]) for k, sub_patch in sub_patches.items()}
    x_recon = np.zeros_like(im_noisy)
    for (i, j), sub_patch_recon in sub_patches_done.items():
        i_max = min((i+1) * 256, im_noisy.shape[0]+1)
        i_min = max(0, i_max - 257)
        j_max = min((j+1) * 256, im_noisy.shape[1]+1)
        j_min = max(0, j_max - 257)
        slice_i = slice(i_min, i_max)
        slice_j = slice(j_min, j_max)
        x_recon[slice_i, slice_j] += sub_patch_recon[0, ..., 0]
    x_recon /= recon_matrix
    return x_recon

In [9]:
mses = [evaluate(noisy_im / 255, im / 255) for im, noisy_im in tqdm_notebook(zip(im_list, noisy_im_list))]

In [10]:
np.mean(mses)

0.050478359922382296

In [11]:
np.mean(-10 * np.log10(mses))

13.466179543653134